In [ ]:
%pip install uce-model

In [2]:
import uce

In [11]:
model = uce.get_pretrained_model(size='small')  # Load the pre-trained model (must have default configuration)
model = model.to('cuda')

Loading small pretrained model with 4 layers...
Using sample 4 layer model


In [21]:
dataset, dataloader = uce.get_processed_dataset(batch_size=32)  # Get default dataset

Using sample AnnData: 10k pbmcs dataset
Using sample 4 layer model
10k_pbmcs_proc already processed. Skipping
PE Idx, Chrom and Starts files already created


In [22]:
import deepspeed
ds_engine = deepspeed.init_inference(model)
model: uce.model.TransformerModel = ds_engine.module

[2024-02-17 20:17:13,930] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.13.2, git-hash=unknown, git-branch=unknown
[2024-02-17 20:17:13,932] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


In [23]:
import torch
from tqdm.auto import tqdm, trange
num_batches = len(dataloader)
with torch.no_grad():
    for i, batch in zip(trange(num_batches), dataloader):
        batch_sentences, mask, idcs, _cell_sentences = batch
        print(batch_sentences.device)
        batch_sentences = batch_sentences.to('cuda')
        batch_sentences = model.pe_embedding(batch_sentences.permute(1, 0).long())
        _, embeddings = model(batch_sentences, mask=mask.half().cuda())
        print(embeddings)

  0%|          | 0/375 [00:00<?, ?it/s]

cpu
tensor([[ 0.0252,  0.0121, -0.0340,  ...,  0.0025,  0.0121, -0.0394],
        [ 0.0140, -0.0183, -0.0172,  ...,  0.0019,  0.0110, -0.0184],
        [ 0.0183,  0.0048, -0.0041,  ..., -0.0061,  0.0223,  0.0064],
        ...,
        [ 0.0070, -0.0136,  0.0258,  ...,  0.0120,  0.0268,  0.0017],
        [ 0.0123,  0.0060,  0.0183,  ...,  0.0174,  0.0237,  0.0065],
        [-0.0053,  0.0234,  0.0004,  ...,  0.0102,  0.0135, -0.0270]],
       device='cuda:0', dtype=torch.float16)
cpu
tensor([[ 0.0081,  0.0119, -0.0126,  ...,  0.0128, -0.0086, -0.0274],
        [ 0.0330, -0.0205, -0.0405,  ...,  0.0360,  0.0072, -0.0300],
        [ 0.0372,  0.0172, -0.0037,  ...,  0.0243, -0.0334, -0.0267],
        ...,
        [ 0.0165, -0.0046,  0.0115,  ...,  0.0003,  0.0201, -0.0050],
        [ 0.0364, -0.0084,  0.0126,  ...,  0.0178,  0.0310, -0.0323],
        [ 0.0014,  0.0082,  0.0020,  ...,  0.0174, -0.0010, -0.0281]],
       device='cuda:0', dtype=torch.float16)
cpu
tensor([[ 0.0259,  0.0107, -0.

KeyboardInterrupt: 